# Microsoft GraphRAG on Databricks

このノートブックでは、MicrosoftのGraphRAGをDatabricks上で実行する方法を説明します。

## GraphRAGとは

GraphRAGは、ナレッジグラフを活用したRAG(Retrieval-Augmented Generation)手法です。
従来のベクトル検索ベースのRAGと異なり、以下の特徴があります:

- **エンティティとリレーションシップの抽出**: テキストから人物、組織、技術などのエンティティと、それらの関係性を抽出
- **コミュニティ検出**: 関連するエンティティをクラスタリングし、階層的なコミュニティ構造を構築
- **Global Search**: コミュニティレポートを活用し、データセット全体に関する高レベルな質問に回答
- **Local Search**: 特定のエンティティに関する詳細な質問に回答

## 前提条件

- Databricks Runtime 14.0 ML以上
- **クラスター**(サーバレスではなく)を使用(LanceDBのファイル操作制約のため)
- OpenAI APIキー(Databricks Secretsに格納済み)
- Unity Catalogが有効なワークスペース

## 参考リンク

- [GraphRAG GitHub](https://github.com/microsoft/graphrag)
- [GraphRAG Documentation](https://microsoft.github.io/graphrag/)

## 1. パッケージのインストール

GraphRAGと可視化用のパッケージをインストールします。

In [0]:
%pip install graphrag pyvis --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-engineering 0.12.1 requires azure-cosmos==4.3.1, but you have azure-cosmos 4.14.3 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

## 2. 作業ディレクトリの設定

GraphRAGはLanceDBを使用してベクトルデータを保存しますが、LanceDBの`rename`操作は
Unity CatalogボリュームやWorkspaceファイルシステムでサポートされていません。
そのため、**ローカルディスク(`/tmp`)** を作業ディレクトリとして使用します。

処理完了後、結果はDelta Tableに保存して永続化します。

In [0]:
import os
import shutil

# Unity Catalog設定
CATALOG = "takaakiyayoi_catalog"
SCHEMA = "default"

# 作業ディレクトリ(クラスターのローカルディスク)
WORK_DIR = "/tmp/graphrag_work"
INPUT_DIR = f"{WORK_DIR}/input"
OUTPUT_DIR = f"{WORK_DIR}/output"
CACHE_DIR = f"{WORK_DIR}/cache"

# 既存のディレクトリがあればクリア
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

# ディレクトリ作成
for d in [WORK_DIR, INPUT_DIR, OUTPUT_DIR, CACHE_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"作業ディレクトリ: {WORK_DIR}")

作業ディレクトリ: /tmp/graphrag_work


## 3. サンプルデータの準備

GraphRAGのインデックス作成には、テキストデータが必要です。
ここではDatabricksエコシステムに関するサンプルテキストを使用します。

実際の利用では、このディレクトリに分析したいテキストファイル(.txt)を配置してください。

In [0]:
sample_text = """
Databricksは、データエンジニアリング、データサイエンス、機械学習のための統合プラットフォームです。
Apache Sparkの創設者たちによって設立され、レイクハウスアーキテクチャを提唱しています。

Unity Catalogは、Databricksのデータガバナンスソリューションです。
データ、MLモデル、AIアセットを一元管理し、きめ細かなアクセス制御を提供します。
Unity CatalogはDatabricksワークスペース全体でメタデータを共有できます。

MLflowは、機械学習のライフサイクル管理のためのオープンソースプラットフォームです。
実験追跡、モデル登録、デプロイメントなどの機能を提供します。
MLflow 3では、LoggedModelという新しい概念が導入され、GenAIアプリケーションの管理が強化されました。

Delta Lakeは、データレイクに信頼性をもたらすオープンソースのストレージレイヤーです。
ACIDトランザクション、スキーマエンフォースメント、タイムトラベルなどの機能を提供します。
DatabricksではDelta Lakeがデフォルトのテーブル形式として使用されています。

Mosaic AIは、Databricksの生成AIソリューションです。
Foundation Modelのファインチューニング、RAGアプリケーション構築、AIエージェント開発をサポートします。
Agent Frameworkを使用すると、本番環境向けのAIエージェントを構築できます。
"""

with open(f"{INPUT_DIR}/databricks_overview.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

print(f"サンプルデータを保存しました: {INPUT_DIR}/databricks_overview.txt")

サンプルデータを保存しました: /tmp/graphrag_work/input/databricks_overview.txt


## 4. OpenAI APIキーの取得

GraphRAGは現在、**OpenAI API**または**Azure OpenAI**が必要です。

**注意**: Databricks Foundation Model API(FMAPI)は、JSON modeの仕様差異により
GraphRAGのクエリ機能と互換性がありません。

In [0]:
# Databricks SecretsからOpenAI APIキーを取得
OPENAI_API_KEY = dbutils.secrets.get(scope="demo-token-takaaki.yayoi", key="openai_api_key")
print("OpenAI APIキーを取得しました")

OpenAI APIキーを取得しました


## 5. 設定ファイルの作成

GraphRAGの動作は`settings.yaml`で制御します。
主な設定項目:

- **models**: 使用するLLMとEmbeddingモデル
- **chunks**: テキスト分割の設定
- **entity_extraction**: 抽出するエンティティタイプ
- **community_reports**: コミュニティレポート生成の設定

In [0]:
settings_content = f"""encoding_model: cl100k_base

models:
  default_chat_model:
    type: openai_chat
    api_key: {OPENAI_API_KEY}
    model: gpt-4o
    encoding_model: cl100k_base
    model_supports_json: true
    max_retries: 3
    request_timeout: 180

  default_embedding_model:
    type: openai_embedding
    api_key: {OPENAI_API_KEY}
    model: text-embedding-3-small
    encoding_model: cl100k_base

chunks:
  size: 1200
  overlap: 100
  group_by_columns:
    - id

input:
  type: file
  file_type: text
  base_dir: "{INPUT_DIR}"
  file_encoding: utf-8
  file_pattern: ".*\\\\.txt$$"

cache:
  type: file
  base_dir: "{CACHE_DIR}"

storage:
  type: file
  base_dir: "{OUTPUT_DIR}"

reporting:
  type: file
  base_dir: "{OUTPUT_DIR}"

entity_extraction:
  entity_types:
    - organization
    - person
    - technology
    - concept
  max_gleanings: 1

summarize_descriptions:
  max_length: 500

claim_extraction:
  enabled: false

community_reports:
  max_length: 2000
  max_input_length: 8000

cluster_graph:
  max_cluster_size: 10

embed_graph:
  enabled: false

umap:
  enabled: false

snapshots:
  graphml: true
  raw_entities: true
  top_level_nodes: true
"""

with open(f"{WORK_DIR}/settings.yaml", "w", encoding="utf-8") as f:
    f.write(settings_content)

print(f"設定ファイルを保存しました: {WORK_DIR}/settings.yaml")

設定ファイルを保存しました: /tmp/graphrag_work/settings.yaml


## 6. インデックスの作成

GraphRAGのインデックス作成プロセスでは、以下の処理が行われます:

1. テキストをチャンクに分割
2. LLMでエンティティとリレーションシップを抽出
3. ナレッジグラフを構築
4. コミュニティ検出(Leidenアルゴリズム)
5. 各コミュニティのサマリーレポート生成
6. エンティティ説明のEmbedding生成

このプロセスにはLLMへの複数回のAPI呼び出しが発生するため、数分かかる場合があります。

In [0]:
os.chdir(WORK_DIR)
!graphrag index --root .

2026-01-11 05:27:17.113502: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-11 05:27:17.114301: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 05:27:17.117436: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 05:27:17.126619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768109237.142158    5685 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768109237.14

## 7. 作成されたデータの確認

インデックス作成により、以下のParquetファイルが生成されます:

- `entities.parquet`: 抽出されたエンティティ
- `relationships.parquet`: エンティティ間の関係性
- `communities.parquet`: コミュニティ構造
- `community_reports.parquet`: 各コミュニティのサマリー
- `text_units.parquet`: 分割されたテキストチャンク

In [0]:
import pandas as pd

print("=== 出力ファイル ===")
for f in sorted(os.listdir(OUTPUT_DIR)):
    file_path = f"{OUTPUT_DIR}/{f}"
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path) / 1024
        print(f"  {f}: {size:.1f} KB")

=== 出力ファイル ===
  communities.parquet: 9.5 KB
  community_reports.parquet: 55.9 KB
  context.json: 0.0 KB
  documents.parquet: 13.2 KB
  entities.parquet: 9.0 KB
  graph.graphml: 1.5 KB
  indexing-engine.log: 28.8 KB
  relationships.parquet: 7.4 KB
  stats.json: 1.0 KB
  text_units.parquet: 15.0 KB


### エンティティの確認

抽出されたエンティティ(人物、組織、技術、概念など)を確認します。

In [0]:
entities_df = pd.read_parquet(f"{OUTPUT_DIR}/entities.parquet")
print(f"エンティティ数: {len(entities_df)} 件")
display(entities_df[["title", "type", "description"]])

エンティティ数: 10 件


title,type,description
DATABRICKS,ORGANIZATION,"Databricks is an integrated platform for data engineering, data science, and machine learning, founded by the creators of Apache Spark and advocates of the Lakehouse architecture"
APACHE SPARK,ORGANIZATION,"Apache Spark is a unified analytics engine for large-scale data processing, and its creators founded Databricks"
UNITY CATALOG,ORGANIZATION,"Unity Catalog is Databricks' data governance solution that centralizes management of data, ML models, and AI assets, providing fine-grained access control and metadata sharing across Databricks workspaces"
MLFLOW,ORGANIZATION,"MLflow is an open-source platform for managing the machine learning lifecycle, offering features like experiment tracking, model registration, and deployment, with MLflow 3 introducing the concept of LoggedModel for enhanced GenAI application management"
DELTA LAKE,ORGANIZATION,"Delta Lake is an open-source storage layer that brings reliability to data lakes, providing features like ACID transactions, schema enforcement, and time travel, and is used as the default table format in Databricks"
MOSAIC AI,ORGANIZATION,"Mosaic AI is Databricks' generative AI solution, supporting fine-tuning of Foundation Models, RAG application building, and AI agent development, with the Agent Framework enabling the construction of production-ready AI agents"
FOUNDATION MODEL,EVENT,"Foundation Model refers to a large AI model that can be fine-tuned for specific tasks, supported by Mosaic AI in Databricks"
RAG APPLICATION,EVENT,"RAG Application refers to Retrieval-Augmented Generation applications, which are supported by Mosaic AI in Databricks"
AGENT FRAMEWORK,EVENT,Agent Framework is a tool within Mosaic AI that enables the construction of production-ready AI agents
GENAI,,


### リレーションシップの確認

エンティティ間の関係性を確認します。

In [0]:
relationships_df = pd.read_parquet(f"{OUTPUT_DIR}/relationships.parquet")
print(f"リレーションシップ数: {len(relationships_df)} 件")
display(relationships_df[["source", "target", "description"]])

リレーションシップ数: 9 件


source,target,description
DATABRICKS,APACHE SPARK,Databricks was founded by the creators of Apache Spark
DATABRICKS,UNITY CATALOG,Unity Catalog is a data governance solution provided by Databricks
DATABRICKS,MLFLOW,MLflow is used within Databricks for managing the machine learning lifecycle
DATABRICKS,DELTA LAKE,Delta Lake is used as the default table format in Databricks
DATABRICKS,MOSAIC AI,Mosaic AI is a generative AI solution offered by Databricks
MLFLOW,GENAI,MLflow 3 introduces the concept of LoggedModel to enhance the management of GenAI applications
MOSAIC AI,FOUNDATION MODEL,Mosaic AI supports the fine-tuning of Foundation Models
MOSAIC AI,RAG APPLICATION,Mosaic AI supports the building of RAG applications
MOSAIC AI,AGENT FRAMEWORK,Mosaic AI includes the Agent Framework for building production-ready AI agents


### コミュニティレポートの確認

関連するエンティティのクラスタ(コミュニティ)と、そのサマリーを確認します。

In [0]:
reports_df = pd.read_parquet(f"{OUTPUT_DIR}/community_reports.parquet")
print(f"コミュニティレポート数: {len(reports_df)} 件\n")

for _, row in reports_df.iterrows():
    print(f"=== {row['title']} ===")
    print(f"{row['summary']}\n")

コミュニティレポート数: 3 件

=== Databricks and Its Ecosystem ===
The community is centered around Databricks, a platform for data engineering, data science, and machine learning, founded by the creators of Apache Spark. Key components of this ecosystem include Apache Spark, Unity Catalog, and Delta Lake, each playing a significant role in enhancing Databricks' capabilities. The relationships between these entities highlight Databricks' focus on data governance, reliability, and large-scale data processing.

=== MLflow and GenAI Integration ===
The community is centered around the integration of MLflow, an open-source platform for managing the machine learning lifecycle, with GenAI applications. MLflow 3 introduces the concept of LoggedModel, which enhances the management of GenAI applications, indicating a significant development in the field of machine learning and artificial intelligence.

=== Mosaic AI and Its Ecosystem ===
The community is centered around Mosaic AI, a generative AI solution 

## 8. ナレッジグラフの可視化

抽出されたエンティティとリレーションシップをグラフとして可視化します。
PyVisを使用してインタラクティブなネットワーク図を生成します。

In [0]:
from pyvis.network import Network
import networkx as nx

# NetworkXグラフを作成
G = nx.Graph()

# エンティティをノードとして追加
for _, row in entities_df.iterrows():
    G.add_node(
        row["title"],
        title=row["description"][:200] if pd.notna(row["description"]) else "",
        group=row["type"] if pd.notna(row.get("type")) else "unknown"
    )

# リレーションシップをエッジとして追加
for _, row in relationships_df.iterrows():
    if row["source"] in G.nodes and row["target"] in G.nodes:
        G.add_edge(
            row["source"],
            row["target"],
            title=row["description"] if pd.notna(row["description"]) else ""
        )

print(f"グラフ統計: {G.number_of_nodes()} ノード, {G.number_of_edges()} エッジ")

グラフ統計: 10 ノード, 9 エッジ


In [0]:
# PyVisでインタラクティブな可視化
net = Network(height="600px", width="100%", bgcolor="#ffffff", font_color="black", notebook=True)

# NetworkXグラフをPyVisに変換
net.from_nx(G)

# 物理演算の設定
net.toggle_physics(True)
net.set_options("""
{
  "nodes": {
    "font": {"size": 14},
    "scaling": {"min": 10, "max": 30}
  },
  "edges": {
    "color": {"inherit": true},
    "smooth": {"type": "continuous"}
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.01,
      "springLength": 100
    },
    "solver": "forceAtlas2Based"
  }
}
""")

# HTMLファイルとして保存
graph_html_path = f"{OUTPUT_DIR}/knowledge_graph.html"
net.save_graph(graph_html_path)
print(f"グラフを保存しました: {graph_html_path}")

グラフを保存しました: /tmp/graphrag_work/output/knowledge_graph.html


### グラフの表示

以下のセルでインタラクティブなナレッジグラフを表示します。
ノードをドラッグして移動したり、ホバーして詳細を確認できます。

In [0]:
# HTMLファイルを読み込んで表示
with open(graph_html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

displayHTML(html_content)

<!-- 
-->

## 9. Global Search

Global Searchは、コミュニティレポートを活用して、データセット全体に関する高レベルな質問に回答します。
「このデータセットの主要なテーマは何か?」といった要約的な質問に適しています。

In [0]:
from pathlib import Path
import graphrag.api as api
from graphrag.config.load_config import load_config

# 設定を読み込み
graphrag_config = load_config(Path(WORK_DIR))

# 必要なデータを読み込み
entities = pd.read_parquet(f"{OUTPUT_DIR}/entities.parquet")
communities = pd.read_parquet(f"{OUTPUT_DIR}/communities.parquet")
community_reports = pd.read_parquet(f"{OUTPUT_DIR}/community_reports.parquet")

2026-01-11 05:28:06.880956: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-11 05:28:06.881795: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 05:28:06.885131: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 05:28:06.894348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768109286.910245    5636 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768109286.91

In [0]:
query = "このデータセットで説明されている主要な技術とその関係性を要約してください"

response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query=query,
)

print("=== 質問 ===")
print(query)
print("\n=== 回答 ===")
print(response)

=== 質問 ===
このデータセットで説明されている主要な技術とその関係性を要約してください

=== 回答 ===
### Databricksの統合プラットフォーム

Databricksは、データエンジニアリング、データサイエンス、機械学習のための統合プラットフォームとして機能しています。このプラットフォームは、Apache Spark、Unity Catalog、Delta Lakeなどのツールを統合することで、データ処理能力を向上させています [Data: Reports (0)]。これにより、ユーザーは大規模なデータセットを効率的に処理し、分析することが可能になります。

### Apache Sparkの役割

Apache Sparkは、Databricksの基盤となるコンポーネントであり、大規模データ処理のための統一分析エンジンを提供します。Databricksの創設者がSparkを開発したことから、DatabricksはSparkの能力に深く依存しています [Data: Reports (0)]。Sparkの強力なデータ処理能力は、Databricksのプラットフォーム全体のパフォーマンスを支えています。

### Unity Catalogによるデータガバナンス

Unity Catalogは、Databricksのデータガバナンスソリューションであり、データ、機械学習モデル、AI資産の管理を中央集権化します。これにより、データ操作のコンプライアンスとデータ整合性が確保されます [Data: Reports (0)]。データガバナンスの強化は、企業がデータを安全かつ効率的に管理するために不可欠です。

### Mosaic AIの役割

Mosaic AIは、Databricksの生成AIソリューションとして、Foundation Modelsの微調整、RAGアプリケーションの開発、AIエージェントの構築をサポートする中心的な存在です。これにより、AI開発と展開の能力が強化されます [Data: Reports (2)]。Mosaic AIの導入により、企業はより高度なAIソリューションを迅速に開発することが可能になります。

### MLflowによる機械学習ライフサイクル管理

MLflowは、機械学習ライフサイクル管理のためのオープンソ

[Trace(trace_id=tr-a1affcdfbd8c88f780925cb7b52c46b1), Trace(trace_id=tr-1a45679e60e0025dd12d9b1fb8b69d36)]

## 10. Local Search

Local Searchは、特定のエンティティに関する詳細な質問に回答します。
ベクトル検索でエンティティを特定し、関連するコンテキストを組み合わせて回答を生成します。

In [0]:
text_units = pd.read_parquet(f"{OUTPUT_DIR}/text_units.parquet")
relationships = pd.read_parquet(f"{OUTPUT_DIR}/relationships.parquet")

query_local = "MLflowとは何ですか?どのような機能がありますか?"

response_local, context_local = await api.local_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    text_units=text_units,
    relationships=relationships,
    covariates=None,
    community_level=2,
    response_type="Multiple Paragraphs",
    query=query_local,
)

print("=== 質問 ===")
print(query_local)
print("\n=== 回答 ===")
print(response_local)

=== 質問 ===
MLflowとは何ですか?どのような機能がありますか?

=== 回答 ===
## MLflowとは

MLflowは、機械学習のライフサイクル管理を目的としたオープンソースプラットフォームです。Databricks内で使用されており、機械学習プロジェクトの管理を効率化するためのさまざまな機能を提供しています [Data: Entities (3); Relationships (2)]。

## 主な機能

### 実験追跡

MLflowは、実験の追跡機能を提供します。これにより、異なるモデルのパラメータ、メトリクス、アーティファクトを記録し、比較することが可能です。これにより、データサイエンティストは実験の結果を容易に管理し、最適なモデルを選択するための情報を得ることができます。

### モデル登録

MLflowは、モデルの登録機能も備えています。これにより、モデルのバージョン管理が可能となり、異なるバージョンのモデルを簡単に管理し、必要に応じて特定のバージョンをデプロイすることができます。

### デプロイメント

MLflowは、モデルのデプロイメントをサポートしており、開発環境から本番環境へのスムーズな移行を可能にします。これにより、モデルの実装が迅速かつ効率的に行われ、ビジネスニーズに応じた迅速な対応が可能となります。

### GenAIアプリケーション管理

MLflow 3では、LoggedModelという新しい概念が導入され、GenAIアプリケーションの管理が強化されています。これにより、生成AIアプリケーションの管理がより効率的になり、AIソリューションの開発と展開が容易になります [Data: Entities (3, 9); Relationships (5)]。

MLflowは、これらの機能を通じて、機械学習プロジェクトの全体的な管理をサポートし、データサイエンティストやエンジニアがより効率的に作業できる環境を提供しています。


[Trace(trace_id=tr-94b4a463001486fc75df291e68b6045a), Trace(trace_id=tr-0fecb7cc5dc33d19355d2e8e366a06d1)]

## 11. Delta Tableへの保存

ローカルディスク上のデータはクラスター終了時に消えてしまうため、
Delta Tableに保存して永続化します。

次回以降は、インデックス作成をスキップしてDelta Tableからデータを読み込んでクエリを実行できます。

In [0]:
tables = {
    "graphrag_entities": "entities.parquet",
    "graphrag_relationships": "relationships.parquet",
    "graphrag_community_reports": "community_reports.parquet",
    "graphrag_communities": "communities.parquet",
    "graphrag_text_units": "text_units.parquet",
}

for table_name, file_name in tables.items():
    file_path = f"file:{OUTPUT_DIR}/{file_name}"
    spark.read.parquet(file_path).write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.{table_name}")
    print(f"✓ {CATALOG}.{SCHEMA}.{table_name}")

✓ takaakiyayoi_catalog.default.graphrag_entities
✓ takaakiyayoi_catalog.default.graphrag_relationships
✓ takaakiyayoi_catalog.default.graphrag_community_reports
✓ takaakiyayoi_catalog.default.graphrag_communities
✓ takaakiyayoi_catalog.default.graphrag_text_units


## 12. (オプション) Delta Tableからの読み込み

保存したDelta Tableからデータを読み込んでクエリを実行する例です。

In [0]:
# # Delta Tableからデータを読み込む場合
# entities = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_entities").toPandas()
# communities = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_communities").toPandas()
# community_reports = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_community_reports").toPandas()
# text_units = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_text_units").toPandas()
# relationships = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_relationships").toPandas()

## まとめ

このノートブックでは、以下を実行しました:

1. **インデックス作成**: テキストからエンティティ、リレーションシップ、コミュニティを抽出
2. **可視化**: ナレッジグラフをインタラクティブに表示
3. **Global Search**: データセット全体に関する質問に回答
4. **Local Search**: 特定のエンティティに関する質問に回答
5. **永続化**: 結果をDelta Tableに保存

### 制約事項

- **LLM**: OpenAI APIまたはAzure OpenAIが必要(FMAPIは非対応)
- **ストレージ**: LanceDBの制約により、クラスターのローカルディスクを使用
- **コスト**: インデックス作成時にLLMへの多数のAPI呼び出しが発生